In [ ]:
import json
with open('/mnt/ssd2/wangke/dataset/datasets.json', 'r') as f:
    #按行读取数据集
    records = [json.loads(line) for line in f]
    #获取不重复的record["repo"]
    repos = set([record["repo"] for record in records])
    print(len(repos))
    print(repos)

12
{'hyperledger/indy-node', 'Blazemeter/taurus', 'psf/requests', 'spotify/luigi', 'open-mmlab/mmdetection', 'mozilla/pontoon', 'spesmilo/electrum', 'cython/cython', 'qutebrowser/qutebrowser', 'biopython/biopython', 'google/clusterfuzz', 'dmlc/dgl'}


In [25]:
import requests
import json
GITHUB_TOKEN = json.load(open("/home/wangke/model/ContextGenerator/settings.json", encoding='utf-8'))["github_tokens"][0]
def check_github_rate_limit():
    url = "https://api.github.com/rate_limit"
    headers = {'Authorization': f'token {GITHUB_TOKEN}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        remaining = data["rate"]["remaining"]
        reset_time = data["rate"]["reset"]  # Unix 时间戳
        print(f"GitHub API 剩余请求次数: {remaining}")
        print(f"API 限制重置时间 (UTC): {reset_time}")
    else:
        print(f"请求失败，状态码: {response.status_code}")

if __name__ == "__main__":
    check_github_rate_limit()

GitHub API 剩余请求次数: 4762
API 限制重置时间 (UTC): 1742215053


In [22]:
str_lines = [
    "```",
    "             msg_aggregator=self.msg_aggregator,",
    "         )",
    "-    def _initialize_uniswap(self, premium: Optional[Premium]) -> None:",
    "-        self.eth_modules['uniswap'] = Uniswap(",
    "-            ethereum_manager=self.ethereum,",
    "-            database=self.database,",
    "-            premium=premium,",
    "-            msg_aggregator=self.msg_aggregator,",
    "-        )",
    " ",
    "     def get_zerion(self) -> Zerion:",
    "         \"\"\"Returns the initialized zerion. If it's not ready it waits for 5 seconds",
    "         and then times out. This should really never happen",
    "```",
]
# str = "```\n  msg_aggregator=self.msg_aggregator        )-    def _initialize_uniswap(self premium: Optional[Premium]) -> None:-        self.eth_modules[‘uniswap’] = Uniswap(-            ethereum_manager=self.ethereum-            database=self.database-            premium=premium-            msg_aggregator=self.msg_aggregator-        )      def get_zerion(self) -> Zerion:         \\\\Returns the initialized zerion. If it's not ready it waits for 5 seconds         and then times out. This should really never happen```"
# str_lines = str.split('\n')
old_lines = []
new_lines = []
for line in str_lines:
    if line.startswith("-"):
        old_lines.append(line)
    elif line.startswith("+"):
        new_lines.append(line)
    else:
        old_lines.append(line)
        new_lines.append(line)
str = '\n'.join(str_lines)
old = '\n'.join(old_lines)
new = '\n'.join(new_lines)
old

'```\n             msg_aggregator=self.msg_aggregator,\n         )\n-    def _initialize_uniswap(self, premium: Optional[Premium]) -> None:\n-        self.eth_modules[\'uniswap\'] = Uniswap(\n-            ethereum_manager=self.ethereum,\n-            database=self.database,\n-            premium=premium,\n-            msg_aggregator=self.msg_aggregator,\n-        )\n \n     def get_zerion(self) -> Zerion:\n         """Returns the initialized zerion. If it\'s not ready it waits for 5 seconds\n         and then times out. This should really never happen\n```'

In [6]:
import json
import model
#写一个程序读取/mnt/ssd2/wangke/dataset/code_review_automation_sota/valid_instances.xlsx的数据并打印
import pandas as pd
result = []
#处理valid的数据
data = pd.read_excel('/mnt/ssd2/wangke/dataset/code_review_automation_sota/valid_instances.xlsx', header=1)
data = data[data['task'] == 'C&NL2C']
for i in range(len(data)):
    input_code = data.iloc[i]['input_code']
    input_nl = data.iloc[i]['input_nl']
    target = data.iloc[i]['target']
    prompt_for_quality_estimation = model.prompt_for_quality_estimation(input_code,input_nl,target)
    instance = {
        "instruction": prompt_for_quality_estimation,
        "input": "",
        "output": "valid"
    }
    result.append(instance)
print(len(data))
#处理discarded的数据
data = pd.read_excel('/mnt/ssd2/wangke/dataset/code_review_automation_sota/discarded_instances.xlsx', header=1)
data = data[data['task'] == 'C&NL2C']
for i in range(len(data)):
    input_code = data.iloc[i]['input_code']
    input_nl = data.iloc[i]['input_nl']
    target = data.iloc[i]['target']
    prompt_for_quality_estimation = model.prompt_for_quality_estimation(input_code,input_nl,target)
    instance = {
        "instruction": prompt_for_quality_estimation,
        "input": "",
        "output": "discarded"
    }
    result.append(instance)
print(len(data))
#打乱result,并划分训练集，验证集，测试集分别为85%，7.5%，7.5%
import random
random.shuffle(result)

train_data = result[:int(len(result)*0.925)]
test_data = result[int(len(result)*0.925):]
with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/train.json', 'w') as f:
    json.dump(train_data, f)
with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/test.json', 'w') as f:
    json.dump(test_data, f)
# train_data = result[:int(len(result)*0.85)]
# valid_data = result[int(len(result)*0.85):int(len(result)*0.925)]
# test_data = result[int(len(result)*0.925):]
# #保存数据
# with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/train_data.json', 'w') as f:
#     json.dump(train_data, f)
# with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/valid_data.json', 'w') as f:
#     json.dump(valid_data, f)
# with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/test_data.json', 'w') as f:
#     json.dump(test_data, f)

739
230


In [8]:
###评估test_dataset 微调后
import json
# with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/test.json", "r") as f:
with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/train.json", "r") as f:
    total = 0
    correct = 0
    records = json.load(f)
    for record in records:
        output = record["output"]
        model_output = record["model_output"]
        if model_output.strip().startswith("valid") or model_output.strip().startswith("Valid"): model_output = "valid"
        elif model_output.strip().startswith("discarded") or model_output.strip().startswith("Discarded"): model_output = "discarded"
        else: print(f"Invalid model output: {model_output}")
        if output == model_output:
            correct += 1
        total += 1
    print(f"Accuracy: {correct/total}")

    

Invalid model output: ```
valid
```



Final Answer: The final answer is valid. I hope it is correct.<|end_of_text|>
Invalid model output: ```
valid
```



Final Answer: The final answer is "valid". I hope it is correct. Let me know if you have any further questions.<|end_of_text|>
Invalid model output: ```
valid
```



Final Answer: The final answer is "valid". I hope it is correct. Let me know if you have any further questions.<|end_of_text|>
Invalid model output: ```
valid
```  Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

Task:
You are an experienced code quality evaluator. Classify the given dataset instance (Old code, Review, New code) as valid or discarded based on strict rules.
Classification Criteria (mark as discarded if ANY condition is met):
Unclear Review: The Review is too vague for humans to infer the required change (e.g., "Fix this" without context).
No Change Asked: The Review is not reques

In [6]:
###评估test_dataset 微调前
import model
with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/test.json", "r") as f:
    total = 0
    correct = 0
    records = json.load(f)
    for record in records:
        output = record["output"]
        instruction = record["instruction"]
        _, wofinetune_output = model.get_model_response(instruction)
        record["wofinetune_output"] = wofinetune_output
        if wofinetune_output.strip().startswith("valid") or wofinetune_output.strip().startswith("Valid"): wofinetune_output = "valid"
        elif wofinetune_output.strip().startswith("discarded") or wofinetune_output.strip().startswith("Discarded"): wofinetune_output = "discarded"
        else: print(f"Invalid model output: {wofinetune_output}")
        if output == wofinetune_output:
            correct += 1
        total += 1
    print(f"Accuracy: {correct/total}")
    with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/test_wofinetune.json", "w") as f:
        json.dump(records, f, indent=4)

prompt:

Task:
You are an experienced code quality evaluator. Classify the given dataset instance (Old code, Review, New code) as valid or discarded based on strict rules.
Classification Criteria (mark as discarded if ANY condition is met):
Unclear Review: The Review is too vague for humans to infer the required change (e.g., "Fix this" without context).
No Change Asked: The Review is not requesting any change (e.g., “Awesome work so far, Eli!”)
Ignored Review: Code changes deviate from the Review’s intent (e.g., Review requests a bug fix, but new features are added).
Wrong Linking: Old code has been linked to a wrong New code while mining the dataset.
Evaluation Steps:
Check Review Intent: Does the Review explicitly request a code change? If not (e.g., praise or no actionable request), mark discarded.
Assess Review Clarity: Is the Review specific enough to infer what to modify? If unclear (e.g., "Improve this"), mark discarded.
Verify Modification Alignment: Does the New code directly

In [ ]:
import json
import model
input_file = "/mnt/ssd2/wangke/CR_data/dataset/map_result/dataset_sorted_llama.json"
output_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed.json"
with open(input_file, "r") as f:
    records = json.load(f)
    for record in records:
        old, review = record["old"], record["review"]
        prompt_for_context_requirement = model.prompt_for_context_requirement(old, review)
        _, result_json = model.get_model_response(prompt_for_context_requirement)
        record["prompt_for_context_requirement"] = prompt_for_context_requirement
        record["result_json"] = result_json
with open(output_file, "w") as f:
    json.dump(records, f, indent=4)

prompt:

Task Prompt for Evaluating Code Review Comments:
As an experienced human evaluator, you receive a reviewer's comment on a specific piece of code that requires modification. Your task involves a thoughtful step-by-step approach to consider the proposed changes. Begin by examining the provided code block and the reviewer’s comment to determine if the intended modifications can be achieved by editing the code in-place.
Instructions:Review the Comment and Code Block:
Comment Provided: Is the name "head" a convention for health checking? Regardless it caught me by surprise, maybe add some docs to this function on why it exist? It should also say what 204.
Associated Code Block:
```
         self.redirect("/static/visualiser/index.html")
     def head(self):
         self.set_status(204)
         self.finish()
```
Make Two Key Judgments:
1.Feasibility: Determine if the proposed changes are feasible within the given code block alone. Consider if the comment:
References elements not i

In [2]:
import json
import model
import re
input_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed.json"
output_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed_processed.json"
with open(input_file, "r") as f:
    records = json.load(f)
    _records = []
    total = 0
    f0a0 = 0
    f0a1 = 0
    f1a0 = 0
    f1a1 = 0
    error = 0
    for record in records:
        try:
            result_json = record["result_json"]
            #采用正则表达式截取result_json中Feasibility后出现的第一个数字,先匹配Feasibility,再匹配第一个数字
            feasibility = re.search(r'"Feasibility": "(\d)"', result_json).group(1)
            additional_context_required = re.search(r'"Additional_context_required": "(\d)"', result_json).group(1)
            # feasibility = re.search(r'"Feasibility": "(\d)"', result_json).group(1)
            # additional_context_required = re.search(r'"Additional_context_required": "(\d)"', result_json).group(1)
            if feasibility == "1" and additional_context_required == "1":
                # _records.append(record)
                f1a1 += 1
            elif feasibility == "1" and additional_context_required == "0": 
                f1a0 += 1
            elif feasibility == "0" and additional_context_required == "1": 
                f0a1 += 1
                _records.append(record)
            elif feasibility == "0" and additional_context_required == "0": 
                f0a0 += 1
                _records.append(record)
            else: error += 1
            total += 1
        except:
            pass
    print(f"total: {total}, f1a1: {f1a1}, f1a0: {f1a0}, f0a1: {f0a1}, f0a0: {f0a0}, error: {error}")
    with open(output_file, "w") as f1:
        json.dump(_records, f1, indent=4)

total: 4422, f1a1: 1, f1a0: 4235, f0a1: 186, f0a0: 0, error: 0


In [9]:
import json
import model
import re
# input_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed_map_deepseek.json"
# output_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed_map_deepseek_processed.json"
input_file = "/mnt/ssd2/wangke/dataset/cr_data/new_datasets_instructed_map_deepseek.json"
output_file = "/mnt/ssd2/wangke/dataset/cr_data/new_datasets_instructed_map_deepseek_processed.json"
with open(input_file, "r") as f:
    records = [json.loads(line) for line in f]
    _records = []
    total = 0
    a1 = 0
    a0 = 0
    error = 0
    for record in records:
        try:
            result_json = '\n'.join(record["repository_context_result_json"])
            #采用正则表达式截取result_json中Feasibility后出现的第一个数字,先匹配Feasibility,再匹配第一个数字
            # additional_context_required = re.search(r'"Additional_context_required": "(\d)"', result_json).group(1)
            additional_context_required = re.search(r'(\d+)', result_json.split("Additional_context_required")[1]).group(0)
            if additional_context_required == "1":
                _records.append(record)
                a1 += 1
            elif additional_context_required == "0": 
                # _records.append(record)
                a0 += 1
            else: error += 1
            total += 1
        except:
            pass
    print(f"total: {total}, f1: {a1}, f0: {a0}, error: {error}")
    with open(output_file, "w") as f1:
        json.dump(_records, f1, indent=4)

total: 8678, f1: 5581, f0: 3097, error: 0


In [ ]:
import re
result_json = [
                "{\"Summary\": \"Importing `json` enables handling of structured data, parsing JSON files, and serializing data. It provides methods for encoding, decoding, and validating JSON data, ensuring proper formatting and security through features like `json.dumps()` with `indent` and `ensure_ascii`, while maintaining compatibility with different text encodings.\"}",
            ]
result_json = '\n'.join(result_json)
additional_context_required = re.search(r'(\d+)', result_json.split("Additional_context_required")[1]).group(0)
element_name_to_retrieve = re.search(r'[a-zA-Z_]+', result_json.split("Element_name_to_retrieve")[1]).group(0)
details_to_retrieve = re.search(r'"Details_to_retrieve"\s*:\s*"((?:[^"\\]|\\.)*)"', result_json).group(1)
print(additional_context_required)
print(element_name_to_retrieve)
print(details_to_retrieve)

Importing `json` enables handling of structured data, parsing JSON files, and serializing data. It provides methods for encoding, decoding, and validating JSON data, ensuring proper formatting and security through features like `json.dumps()` with `indent` and `ensure_ascii`, while maintaining compatibility with different text encodings.


In [1]:
line = "dfasdjfklasdhlkf"
print(line.index("jfk"))

5


In [7]:
import re
result_json = [
                "{\"Summary\": \"Importing `json` enables handling of structured data, parsing JSON files, and serializing data. It provides methods for encoding, decoding, and validating JSON data, ensuring proper formatting and security through features like `json.dumps()` with `indent` and `ensure_ascii`, while maintaining compatibility with different text encodings.\"}",
            ]
result_json = '\n'.join(result_json)

def get_json_value_string(str, key):
        try:
            return re.search(rf'"{key}"\s*:\s*"((?:[^"\\]|\\.)*)"', str).group(1)
        except:
            return ""
        
str = get_json_value_string(result_json, "Summary")
print(str)

Importing `json` enables handling of structured data, parsing JSON files, and serializing data. It provides methods for encoding, decoding, and validating JSON data, ensuring proper formatting and security through features like `json.dumps()` with `indent` and `ensure_ascii`, while maintaining compatibility with different text encodings.


In [ ]:
import json
import re
import traceback
config = {
    "dataset_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/datasets/CR_and_CRN.json",
    "output_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/datasets/CR_and_CRN_estimated.json"
}

def normalize_text(text):
    text = re.sub(r'\W+','', text)
    return text

with open(config["dataset_path"], "r") as f:
    records = [json.loads(line) for line in f]
    for record in records:
        try:
            if record["review"].find("suggestion") != -1:#含有suggestion的不太可能需要上下文
                continue

            if record["repo_context_dependency_estimation"]["Additional_context_required"] == "0":
                continue
            old = record["old"]

            comment = record["comment"]
            diff_hunk_lines = comment["diff_hunk"].split('\n')
            start = int(re.findall(r'(\d+)', diff_hunk_lines[0])[2])
            if comment["original_start_line"] and comment["original_start_line"]-start+1 < len(diff_hunk_lines):
                comment["review_hunk_start_line"] = diff_hunk_lines[comment["original_start_line"]-start+1][1:] #加1是因为第一行是code_diff_hunk的prefix
            index = len(diff_hunk_lines)-1 # 指向review_position_line
            for i in range(index, -1, -1):
                line = diff_hunk_lines[i][1:]
                if line:
                    comment["review_position_line"] = line
                    break

            review_line = record["comment"]["review_position_line"]
            flag = False
            for line in old.split("\n"):
                if normalize_text(line) == normalize_text(review_line):
                    flag = True
                    break
            if not flag: continue
            with open(config["output_path"], "a") as f0:
                f0.write(json.dumps(record, ensure_ascii=False) + "\n")
        except Exception as e:
             print(e)
             traceback.print_exc()

In [28]:
import json
config = {
    "dataset_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/datasets/CR_and_CRN_4_6.json",
    # "dataset_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/result_4_6.json",
    "output_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/tmp_result.json"
}
with open(config["dataset_path"], "r") as f:
    records = [json.loads(line) for line in f]
    print(len(records))
    record = records[11]

with open(config["output_path"], "w") as f1:
    json.dump(record, f1, indent=4)

628


In [ ]:
import json
from getProjectCommitState import CLBPP

config = {
    "dataset_path": "/mnt/ssd2/wangke/dataset/datasets.json",
    "output_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/datasets/new_datasets.json"
}

with open(config["dataset_path"], "r") as f:
    records = [json.loads(line) for line in f]
    for record in records:
        if record["_id"] <= 1937: continue
        try:
            record = CLBPP(record)
        except Exception as e:
            id = record["_id"]
            print(f"Error in processing {id}")
            print(e)
            continue
        with open(config["output_path"], "a") as f:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

# with open(config["output_path"], "r") as f:
#     records = [json.loads(line) for line in f]
#     print(len(records))
#     print(records[-1]["_id"])

In [1]:
import json
from ContextGenerators.LanguageContextGeneratorManager import LanguageContextGenerator
import getProjectCommitState
from getProjectCommitState import CLBPP
import model

config = {
    # "dataset_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/datasets/new_repo_datasets_estimated.json",
    "dataset_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/datasets/new_repo_datasets.json",
    #  "dataset_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/datasets/new_datasets.json",
    "output_path": "/mnt/ssd2/wangke/dataset/AgentRefiner/tmp_result.json"
}
with open(config["dataset_path"], "r") as f:
    records = [json.loads(line) for line in f]
    print(len(records))
    # print(records[5]["_id"])
    record = records[7]
    # record["old"] = record["old"].split("\n")
    # record["new"] = record["new"].split("\n")
    record = CLBPP(record)
    languageContextGenerator = LanguageContextGenerator(record)
    contextGenerator = languageContextGenerator.context_generator
    defs_bef = contextGenerator.getContext()
    print(defs_bef)
    


# with open(config["output_path"], "w") as f1:
#     json.dump(record, f1, indent=4)

15100


fatal: reference is not a tree: a99a729c6b62980a5d2dfaa4960f6d067de381f6


git checkout a99a729c6b62980a5d2dfaa4960f6d067de381f6 failed. Fetching parent commit.


fatal: reference is not a tree: 351e83b93a4ec6daecbcef4584ea5bde4531308e


git checkout 351e83b93a4ec6daecbcef4584ea5bde4531308e failed. Fetching parent commit.


...

git checkout 6991fb0dbf8be5263207287a7677c7ac1b8ddd05 failed. Fetching parent commit.
Successfully checked out to commit 6c92c5a539276d387b85eedc89be1f888962647d after 10 searches
Untracked files and dirs cleaned


Note: switching to '6c92c5a539276d387b85eedc89be1f888962647d'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 6c92c5a5 [pre-commit.ci] pre-commit autoupdate (#12542)


Applied patch for machine_learning/ridge_regression.py commit_info:6991fb0dbf8be5263207287a7677c7ac1b8ddd05
Applied patch for machine_learning/ridge_regression.py commit_info:fb1b7a72d0dc9e68948d536dfba3973db044c37d
Applied patch for machine_learning/ridge_regression.py commit_info:b7f49aeda933137c4a387a8ef9f79d854977cbb0
Applied patch for machine_learning/ridge_regression.py commit_info:2722754fea454382301fe5a20d7c6319cb54ba54
Applied patch for machine_learning/ridge_regression.py commit_info:106b6c7c571247a2c5f9654aa2034fcdbcc9f7c5
Applied patch for machine_learning/ridge_regression.py commit_info:c110a125f9096ff9c4d758368388a905519427c9
Applied patch for machine_learning/ridge_regression.py commit_info:d5646985a2ac9adeec16fc024c817383c88a6291
Applied patch for machine_learning/ridge_regression.py commit_info:ffd18e02af436542d098107ed0220df24bbe7126
Applied patch for machine_learning/ridge_regression.py commit_info:351e83b93a4ec6daecbcef4584ea5bde4531308e
Applied patch for machine_le

ERROR:tornado.general:SEND Error: Host unreachable


Find definition: numpy in /home/wangke/venv/alpaca-lora/lib/python3.10/site-packages/numpy/__init__.py
Find definition: numpy in /home/wangke/venv/alpaca-lora/lib/python3.10/site-packages/numpy/__init__.py
Find definition: matplotlib in /home/wangke/venv/alpaca-lora/lib/python3.10/site-packages/matplotlib/__init__.py
Find definition: pyplot in /home/wangke/venv/alpaca-lora/lib/python3.10/site-packages/matplotlib/pyplot.py
Find definition: pyplot in /home/wangke/venv/alpaca-lora/lib/python3.10/site-packages/matplotlib/pyplot.py
No definition found in /mnt/ssd2/wangke/CR_data/repo/Python/machine_learning/ridge_regression.py at (4,6)
No definition found in /mnt/ssd2/wangke/CR_data/repo/Python/machine_learning/ridge_regression.py at (4,21)
[{'name': 'numpy', 'path': 'numpy', 'type': 'module', 'text': '"""\nNumPy\n=====\n\nProvides\n  1. An array object of arbitrary homogeneous items\n  2. Fast mathematical operations over arrays\n  3. Linear Algebra, Fourier Transforms, Random Number Gener

In [3]:
old = [
    "",
    "-",
    " import numpy as np",
    " from matplotlib import pyplot as plt",
    " from sklearn import datasets",
    " ",
    " # Ridge Regression function",
    " # reference : https://en.wikipedia.org/wiki/Ridge_regression",
    "-def ridge_cost_function(x: np.ndarray, y: np.ndarray, theta: np.ndarray, alpha: float) -> float:",
    "     \"\"\"",
    "     Compute the Ridge regression cost function with L2 regularization.",
    " ",
    ""
]

new = [
    "",
    " import numpy as np",
    " from matplotlib import pyplot as plt",
    " from sklearn import datasets",
    " ",
    "+",
    " # Ridge Regression function",
    " # reference : https://en.wikipedia.org/wiki/Ridge_regression",
    "+def ridge_cost_function(",
    "+    x: np.ndarray, y: np.ndarray, theta: np.ndarray, alpha: float",
    "+) -> float:",
    "     \"\"\"",
    "     Compute the Ridge regression cost function with L2 regularization.",
    " ",
    ""
]

def merge_diff(old, new):
    diff = []
    i, j = 0, 0
    while i < len(old) or j < len(new):
        line_old = old[i] if i < len(old) else ""
        line_new = new[j] if j < len(new) else ""

        if line_old.startswith("-"):
            diff.append(line_old)
            i += 1
        elif line_new.startswith("+"):
            diff.append(line_new)
            j += 1
        else:
            # Prefer new line if available and not a + line
            if j < len(new) and not line_new.startswith("+"):
                diff.append(line_new)
            elif i < len(old):
                diff.append(line_old)
            i += 1
            j += 1

    return diff

print(merge_diff(old, new))

['', '-', ' import numpy as np', ' from matplotlib import pyplot as plt', ' from sklearn import datasets', ' ', '+', ' # Ridge Regression function', ' # reference : https://en.wikipedia.org/wiki/Ridge_regression', '-def ridge_cost_function(x: np.ndarray, y: np.ndarray, theta: np.ndarray, alpha: float) -> float:', '+def ridge_cost_function(', '+    x: np.ndarray, y: np.ndarray, theta: np.ndarray, alpha: float', '+) -> float:', '     """', '     Compute the Ridge regression cost function with L2 regularization.', ' ', '']
